In [2]:
import wandb
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from dotenv import load_dotenv

load_dotenv("../.env")

True

In [3]:
########################################################################################
# query data from wandb

# add data to frames
api = wandb.Api()

data_description_map = {
    "vox2": "voxceleb2 (1M files shared between 5994 speakers)",
    "tiny-few": "tiny-few (50k files shared between 100 speakers)",
    "tiny-low": "tiny-low (8 files from 1 session for 5994 speakers)",
    "tiny-high": "tiny-high (8 files from 8 sessions for 5994 speakers)",
}


def load_runs(name: str):
    runs = api.runs(name)
    df = pd.DataFrame(columns=["num_steps", "eer", "data", "network"])

    for r in runs:
        tags = r.tags

        if r.state != "finished":
            continue

        if (
            "step_exp" in tags
        ):
            num_steps = r.config["trainer"]["max_steps"]
            eer = r.summary["test_eer_hard"]

            if eer >= 1:
                continue

            if "vox2_full" in tags:
                data = data_description_map["vox2"]
            elif "tiny_few" in tags:
                data = data_description_map["tiny-few"]
            elif "tiny_many_high" in tags:
                data = data_description_map["tiny-high"]
            elif "tiny_many_low" in tags:
                data = data_description_map["tiny-low"]
            elif "tiny_many" in tags:
                continue
            else:
                raise ValueError(f"undetermined dataset from {tags=}")

            df = pd.concat(
                [
                    df,
                    pd.DataFrame(
                        {
                            "num_steps": [num_steps],
                            "eer": [eer],
                            "data": [data],
                            "network": [name],
                        }
                    ),
                ],
                ignore_index=True,
            )

    return df


df_xvector = load_runs("xvector")
df_ecapa = load_runs("ecapa")
df_w2v2 = load_runs("wav2vec2")

df = pd.concat([df_xvector, df_ecapa, df_w2v2], ignore_index=True)

In [4]:
df

,num_steps,eer,data,network
0,25000,0.284384,tiny-low (8 files from 1 session for 5994 spea...,xvector
1,400000,0.241335,tiny-low (8 files from 1 session for 5994 spea...,xvector
2,400000,0.240692,tiny-low (8 files from 1 session for 5994 spea...,xvector
3,400000,0.237838,tiny-low (8 files from 1 session for 5994 spea...,xvector
4,100000,0.266305,tiny-low (8 files from 1 session for 5994 spea...,xvector
...,...,...,...,...
136,25000,0.224425,tiny-few (50k files shared between 100 speakers),wav2vec2
137,25000,0.077285,voxceleb2 (1M files shared between 5994 speakers),wav2vec2
138,25000,0.238626,tiny-low (8 files from 1 session for 5994 spea...,wav2vec2
139,25000,0.109838,tiny-high (8 files from 8 sessions for 5994 sp...,wav2vec2


In [5]:
df['network'].unique()


array(['xvector', 'ecapa', 'wav2vec2'], dtype=object)

In [6]:
df_grouped = df.groupby(by=["network", "data", "num_steps"])

In [7]:
df_agg = df_grouped.agg(
    num_steps=("num_steps", "first"),
    eer_min=("eer", "min"),
    eer_max=("eer", "max"),
    eer_mean=("eer", "mean"),
    eer_std=("eer", "std"),
    count=("eer", "count")
)

In [8]:
df_agg

num_steps  \
network  data                                               num_steps             
ecapa    tiny-few (50k files shared between 100 speakers)   25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         tiny-high (8 files from 8 sessions for 5994 spe... 25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         tiny-low (8 files from 1 session for 5994 speak... 25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         voxceleb2 (1M files shared between 5994 speakers)  25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
wav2vec2 tiny-few (50k files shared between 100 speakers)   25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         tiny-high (8 files from 8 sessions for 5994 spe... 25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         tiny-low (8 files from 1 session for 5994 speak... 25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         voxceleb2 (1M files shared between 5994 speakers)  25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
xvector  tiny-few (50k files shared between 100 speakers)   25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         tiny-high (8 files from 8 sessions for 5994 spe... 25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         tiny-low (8 files from 1 session for 5994 speak... 25000         25000   
                                                            50000         50000   
                                                            100000       100000   
                                                            400000       400000   
         voxceleb2 (1M files shared between 5994 speakers)  25000         25000   
                                                            50000         50000   
                                                            100000       100000   
   

In [9]:
df_agg['count'].sum()

141